# CategoricalChangepoint2D Model Demo

This notebook demonstrates the usage of the CategoricalChangepoint2D model with dummy data.

In [ ]:
import numpy as np
# Try to import matplotlib, create dummy if not available
try:
    import matplotlib.pyplot as plt
except ImportError:
    print('matplotlib not available - creating dummy plt object')
    class DummyPlt:
        def __getattr__(self, name):
            def dummy_func(*args, **kwargs):
                pass
            return dummy_func
    plt = DummyPlt()
import sys
import os

# Add pytau to path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), '..', '..'))

from pytau.changepoint_model import CategoricalChangepoint2D, gen_test_array

## Generate Test Data

In [ ]:
# Generate dummy data with obvious changepoints for categorical model
np.random.seed(42)
n_trials = 10
n_time = 100
n_states = 3
n_categories = 4

# Create data with obvious state transitions in category preferences
# State 1: prefers category 0
# State 2: prefers category 2
# State 3: prefers category 1
state_probs = [
    [0.7, 0.1, 0.1, 0.1],  # State 1: mostly category 0
    [0.1, 0.1, 0.7, 0.1],  # State 2: mostly category 2
    [0.1, 0.7, 0.1, 0.1]   # State 3: mostly category 1
]
transition_points = [33, 66]  # Clear transition points

test_data = np.zeros((n_trials, n_time), dtype=int)
for trial in range(n_trials):
    # Add some trial-to-trial variability in transition points
    trial_transitions = [t + np.random.randint(-3, 4) for t in transition_points]
    trial_transitions = [max(5, min(n_time-5, t)) for t in trial_transitions]  # Keep within bounds
    
    # Generate data for each state
    test_data[trial, :trial_transitions[0]] = np.random.choice(n_categories, size=trial_transitions[0], p=state_probs[0])
    test_data[trial, trial_transitions[0]:trial_transitions[1]] = np.random.choice(n_categories, size=trial_transitions[1] - trial_transitions[0], p=state_probs[1])
    test_data[trial, trial_transitions[1]:] = np.random.choice(n_categories, size=n_time - trial_transitions[1], p=state_probs[2])

print(f"Generated test data shape: {test_data.shape}")
print(f"Data range: [{test_data.min()}, {test_data.max()}]")
print(f"Data type: {test_data.dtype}")
print(f"Unique categories: {np.unique(test_data)}")
print(f"State probabilities: {state_probs}")
print(f"Transition points (approx): {transition_points}")

## Initialize and Fit Model

In [ ]:
# Initialize model
model = CategoricalChangepoint2D(
    data_array=test_data,
    n_states=n_states,
    fit_type='vi'
)

# Generate the PyMC model
model.generate_model()
print("Model generated successfully")

## Run Model Test

In [ ]:
# Test the model
try:
    model.test()
    print("Model test completed successfully")
except Exception as e:
    print(f"Model test failed: {e}")

## Visualize Input Data

In [ ]:
# Plot the input data
plt.figure(figsize=(15, 8))

# Plot time series for first few trials
plt.subplot(2, 2, 1)
for trial in range(min(5, test_data.shape[0])):
    plt.plot(test_data[trial], 'o-', markersize=2, alpha=0.7, label=f'Trial {trial+1}')
plt.title('Categorical Time Series (First 5 Trials)')
plt.xlabel('Time')
plt.ylabel('Category')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot category distribution
plt.subplot(2, 2, 2)
plt.hist(test_data.flatten(), bins=range(n_categories+1), alpha=0.7, edgecolor='black')
plt.title('Category Distribution')
plt.xlabel('Category')
plt.ylabel('Frequency')
plt.xticks(range(n_categories))
plt.grid(True, alpha=0.3)

# Plot heatmap of all trials
plt.subplot(2, 1, 2)
plt.imshow(test_data, aspect='auto', cmap='viridis', interpolation='nearest')
plt.colorbar(label='Category')
plt.title('Categorical Data Heatmap (All Trials)')
plt.xlabel('Time')
plt.ylabel('Trial')

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:
1. Generating dummy categorical data
2. Initializing the CategoricalChangepoint2D model
3. Running model tests
4. Visualizing the categorical input data

The CategoricalChangepoint2D model is designed for detecting changepoints in categorical/discrete data where observations can take on one of several discrete values.